# Master Sheet for DraftKings Daily Fantasy Baseball
#### Scrapes Data from FanGraphs every day and spits out projections for the upcoming slate of games.
#### Broken into 4? sections: Player info from FG and DK excel sheet; Pitcher Scrape; Batter Scrape; Z-score and exportation to excel doc
###### Do park factors periodically. 

#### Set up
##### Importing packages

In [ ]:
pip install git+https://

In [1]:
import csv
from urllib.request import urlopen as UReq
import numpy as np
from bs4 import BeautifulSoup as soup
import statistics
import io
import pandas as pd
import itertools
import datetime as dt
import requests
from collections import OrderedDict
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
import math
import time
import datetime as dt
from datetime import timedelta

#### Setting up my dates for urls

In [2]:
today = dt.datetime.today().strftime("%m.%d.%Y")
today_month = int(today[0:2])
today_day = int(today[3:5])
today_year = int(today[6:10])

In [3]:
print("DraftKings projections for: Month:", today_month, "Day:", today_day, "Year:", today_year)

DraftKings projections for: Month: 6 Day: 16 Year: 2021


In [4]:
######################
#####PLAYER INFO#####
#####################

#Reading in the DK Salary Sheet. Start with 0 for the first slate and 1 for the second etc. 
#This way I can track the different slates and will help with analysis down the road

GameSlate_Read = 'Daily Sheets/DKSalaries_6.16.2021_0.csv'
Slate = GameSlate_Read[-5]
DKDailyRead = pd.read_csv(GameSlate_Read)
FGPlayer_Info = pd.read_excel('FGPlayerInfo/FGPlayerInfo_06.16.21.xlsx')
TeamAbbrev = pd.read_excel('Team Abbreviations.xlsx')

###Weather Website####
website = 'https://rotogrinders.com/weather/mlb'

#######################
###URLS FOR PITCHERS###
#######################
pitcher_dashboard = "https://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=all&qual=0&type=8&season=2021&month=0&season1=2021&ind=0&team=0&rost=0&age=0&filter=&players=0&startdate=&enddate="

#Pitcher Splits
vs_hand_std = "https://www.fangraphs.com/leaders/splits-leaderboards?splitArr=HAND&splitArrPitch=&position=P&autoPt=false&splitTeams=false&statType=player&statgroup=1&startDate=2021-03-01&endDate=2021-11-01&players=&filter=&groupBy=career&sort=-1,1&pageitems=10000000000000"
# split arr = 5 is LHH, split arr = 6 is RHH
vs_hand_adv = "https://www.fangraphs.com/leaders/splits-leaderboards?splitArr=HAND&splitArrPitch=&position=P&autoPt=false&splitTeams=false&statType=player&statgroup=2&startDate=2021-03-01&endDate=2021-11-01&players=&filter=&groupBy=career&sort=-1,1&pageitems=10000000000000"
pitcher_last_14 = "https://www.fangraphs.com/leaders/splits-leaderboards?splitArr=&splitArrPitch=&position=P&autoPt=false&splitTeams=false&statType=player&statgroup=2&startDate=2021-06-01&endDate=2021-06-15&players=&filter=IP%7Cgt%7C0&groupBy=season&sort=-1,1&pageitems=10000000000000"

######################
###URLS FOR BATTERS###
######################
fg_dashboard = "https://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=0&type=8&season=2021&month=0&season1=2021&ind=0&team=0&rost=0&age=0&filter=&players=0&startdate=&enddate=" # FG dashboard website. Have to use this to toggle to each display
#team_dashboard = "https://www.fangraphs.com/leaders/splits-leaderboards?splitArr=&splitArrPitch=&position=B&autoPt=false&splitTeams=false&statType=team&statgroup=1&startDate=2020-03-01&endDate=2020-11-01&players=&filter=&groupBy=game&sort=-1,1&pageitems=10000000000000"
vs_hand_website_adv = "https://www.fangraphs.com/leaders/splits-leaderboards?splitArr=HAND&splitArrPitch=&position=B&autoPt=false&splitTeams=false&statType=player&statgroup=2&startDate=2021-03-01&endDate=2021-11-01&players=&filter=&groupBy=career&sort=-1,1&pageitems=10000000000000"
# splitArr = 1 is lhp, 2 is rhp
vs_hand_website_std = "https://www.fangraphs.com/leaders/splits-leaderboards?splitArr=HAND&splitArrPitch=&position=B&autoPt=false&splitTeams=false&statType=player&statgroup=1&startDate=2021-03-01&endDate=2021-11-01&players=&filter=&groupBy=career&sort=-1,1&pageitems=10000000000000"
last_7_website = "https://www.fangraphs.com/leaders/splits-leaderboards?splitArr=&splitArrPitch=&position=B&autoPt=false&splitTeams=false&statType=player&statgroup=1&startDate=2021-06-08&endDate=2021-06-15&players=&filter=&groupBy=season&sort=-1,1&pageitems=10000000000000"

#######################
#####BOTH##############
#######################
projected_starters = "https://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=all&qual=0&type=8&season=2020&month=0&season1=2020&ind=0&team=0&rost=0&age=0&filter=&players=p2021-06-16&page=1_50"
Park_Factors = pd.read_excel("ParkFactors.xlsx")

###Teams
team_standard = 'https://www.fangraphs.com/leaders/splits-leaderboards?splitArr=HAND&splitArrPitch=&position=B&autoPt=false&splitTeams=false&statType=team&statgroup=1&startDate=2021-03-01&endDate=2021-11-01&players=&filter=&groupBy=career&sort=-1,1&pageitems=10000000000000&pg=0'
# splitArr = 1 is lhp, 2 is rhp
team_adv = 'https://www.fangraphs.com/leaders/splits-leaderboards?splitArr=HAND&splitArrPitch=&position=B&autoPt=false&splitTeams=false&statType=team&statgroup=2&startDate=2021-03-01&endDate=2021-11-01&players=&filter=&groupBy=career&sort=-1,1&pageitems=10000000000000&pg=0'
team_battedball = 'https://www.fangraphs.com/leaders/splits-leaderboards?splitArr=HAND&splitArrPitch=&position=B&autoPt=false&splitTeams=false&statType=team&statgroup=3&startDate=2021-03-01&endDate=2021-11-01&players=&filter=&groupBy=career&sort=-1,1&pageitems=10000000000000&pg=0'

### Weather and Odds

In [5]:
specialweathernote = []
awayteam = []
hometeam = []
UClient = UReq(website)
page_html = UClient.read()
UClient.close()
page_soup = soup(page_html, "html.parser")
special_container = page_soup.findAll("ul", {"class":"lst data"})
allGame_container = page_soup.findAll("div", {"class":"blk crd"})

for i in range(len(special_container[0].findAll("span", {'class': 'game'}))):
    awayteam.append(special_container[0].findAll("span", {'class': 'game'})[i].text.replace(" ","").split("@")[0].split('\n')[1])
    hometeam.append(special_container[0].findAll("span", {'class': 'game'})[i].text.replace(" ","").split("@")[1].split('\n')[1])
    specialweathernote.append(special_container[0].findAll("span", {'class': 'description'})[i].text)

specialWeather_df = pd.DataFrame({
    'Away Team': awayteam,
    'Home Team': hometeam,
    'Weather Note': specialweathernote,
})

awayteam = []
hometeam = []
overunder = []
temp = []
weatherdesc = []
precip = []
windmph= []
winddir = []

for i in range(len(allGame_container)):
    try:
        awayteam.append(allGame_container[i].findAll("span", {'class': 'shrt'})[0].text)
        hometeam.append(allGame_container[i].findAll("span", {'class': 'shrt'})[1].text)
        overunder.append(allGame_container[i].findAll("span", {'class': 'overunder'})[0].text.replace(" ", "").split("\n")[1])
        temp.append(allGame_container[i].findAll("span", {'class': 'value'})[0].text)
        weatherdesc.append(str(allGame_container[i].findAll("div", {'class': ''})[0]).split("display icn-")[1].split('"')[0])
        precip.append(allGame_container[i].findAll("span", {'class': 'value'})[1].text)
        windmph.append(allGame_container[i].findAll("span", {'class': 'value'})[3].text)
        winddir.append(str(allGame_container[i].findAll("div", {'class': ''})[1]).split("display icn-")[1].split('"')[0].split("wind-")[1].split("-")[0])
    except:
        temp.append('This Game is Played in a Dome')
        weatherdesc.append('N/A')
        precip.append('N/A')
        windmph.append('N/A')
        winddir.append('N/A')

weatherOdds_df = pd.DataFrame({
    'Away Team': awayteam,
    'Home Team': hometeam,
    'O/U': overunder,
    'Temperature': temp,
    'Weather Description': weatherdesc,
    'Precipitation': precip,
    'Wind MPH': windmph,
    'Wind Park Direction': winddir,
})
weather_merge = weatherOdds_df.merge(specialWeather_df, right_on = ['Away Team', 'Home Team'], left_on= ['Away Team', 'Home Team'], how = 'left')

#### Bring in Daily DK Sheet and Scrape FanGraphs to get player info and FG ID for all the scrapes
##### Remember to update the FG Info every few days.

##### Draft Kings Daily Sheet

In [6]:
away_team = []
home_team = []
for i in range(len(DKDailyRead['Game Info'])): # create the home and away teams based on the game info. Also, drop off any cancelled or postponed games.
    if "@" in DKDailyRead['Game Info'][i]:
        away_team.append(DKDailyRead['Game Info'][i].split("@")[0])
        home_team.append(DKDailyRead['Game Info'][i].split("@")[1].split(" ")[0])
    else:
        away_team.append("drop")
        home_team.append("drop")

DKDailyRead['Home Team'] = home_team
DKDailyRead['Away Team'] = away_team

DKDailyRead_mask =  DKDailyRead['Home Team'] != "drop"
DKDaily_clean = DKDailyRead.loc[DKDailyRead_mask,:].copy().reset_index().drop("index", axis = 1)

##Creating Game Info for a tab on my final sheet. Will help keep track for analysis down the road as well.
DKDaily_GameInfo = DKDaily_clean[['Game Info', 'Home Team', 'Away Team']].drop_duplicates(subset=['Game Info']).reset_index().drop("index", axis = 1)
DKDaily_GameInfo_weatherteam_abbrev_home = DKDaily_GameInfo.merge(TeamAbbrev, right_on= ['Team Abbrev'], left_on = ['Home Team'], how = 'outer').drop(["Mascot", "Full Name", "Zip Code", 'Team Abbrev'],axis = 1)
DKDaily_GameInfo_weatherteam_abbrev_away = DKDaily_GameInfo_weatherteam_abbrev_home.merge(TeamAbbrev, right_on= ['Team Abbrev'], left_on = ['Away Team'], how = 'outer').drop(["Mascot", "Full Name", "Zip Code", "Team Abbrev", 'Home Team', 'Away Team'],axis = 1)
DKDaily_GameInfo = DKDaily_GameInfo_weatherteam_abbrev_away.merge(weather_merge, right_on= ["Home Team", 'Away Team'], left_on = ['Unnamed: 4_x', 'Unnamed: 4_y'], how = 'outer').drop(['Unnamed: 4_x', 'Unnamed: 4_y'], axis=1)

DkDaily_mask = (DKDaily_GameInfo["Game Info"].isna() == False)
DKDaily_GameInfo = DKDaily_GameInfo.loc[DkDaily_mask,:].copy().reset_index()

DKDaily_GameInfo['Date'] = ""
DKDaily_GameInfo['Game Time'] = ""

for i in range(len(DKDaily_GameInfo)):
    DKDaily_GameInfo.loc[i,'Date'] = DKDaily_GameInfo.loc[i,'Game Info'].split(" ")[1]
    DKDaily_GameInfo.loc[i,'Game Time'] = DKDaily_GameInfo.loc[i,'Game Info'].split(" ")[2]
    
DKDaily_GameInfo=DKDaily_GameInfo[['Home Team', 'Away Team', 'Date', 'Game Time', "O/U", "Temperature", "Weather Description", "Precipitation", "Wind MPH", "Wind Park Direction", "Weather Note"]]

In [7]:
## fixing known weird names, i.e. Enrique Hernandez to Kike
for i in range(len(DKDaily_clean)):
    if DKDaily_clean.loc[i,'Name'] == 'Kike Hernandez':
        DKDaily_clean.loc[i,'Name'] =  'Enrique Hernandez'
        

In [8]:
#going to take out all spaces, period, expressions, etc...
mergename = []
for i in range(len(DKDaily_clean)):
    mergename.append(DKDaily_clean['Name'][i].replace("jr.", "").replace(" ", "").replace(".", "").replace("'", "").replace("-", "").lower().replace("켈리",""))
DKDaily_clean['Clean Name'] = mergename

In [9]:
PlayerMap = DKDaily_clean.merge(FGPlayer_Info, right_on = ["CleanName", "Team"], left_on = ["Clean Name", 'TeamAbbrev'], how = "inner").drop(["Player", "Name", "Clean Name", "CleanName", "Pitcher?"], axis =1)
for i in range(len(PlayerMap)):
    PlayerMap.loc[i,"FGID"] = str(PlayerMap.loc[i,"FGID"])

# Team Splits

In [10]:
team = []
wrc = []
ISO = []
hand = []
k= []

for j in range(1,3):
    driver = webdriver.Firefox()
    driver.get(team_adv.replace("HAND", str(j)))
    html = driver.execute_script("return document.documentElement.outerHTML")
    sel_soup = soup(html, 'html.parser')
    player_box = sel_soup.findAll("div", {"class": "table-scroll"})
    player_box = player_box[0].findAll("tr", {"class": ""})
    for i in range(1,len(player_box)):
        link = player_box[i].find("a")
        team.append(player_box[i].find("td", {"data-stat": "Tm"}).text)
        wrc.append(player_box[i].find("td", {"data-stat": "wRC+"}).text)
        ISO.append(player_box[i].find("td", {"data-stat": "ISO"}).text)
        k.append(player_box[i].find("td", {"data-stat": "K%"}).text)
        hand.append(j)
    driver.quit()

team_advanced_splits_df = pd.DataFrame ({
    "Team": team,
    "wRC+": wrc,
    "ISO": ISO,
    "K%": k,
    "Hand": hand,
},
)        


In [11]:
team = []
hand = []
dkpoints = []

for j in range(1,3):
    driver = webdriver.Firefox()
    driver.get(team_standard.replace("HAND", str(j)))
    html = driver.execute_script("return document.documentElement.outerHTML")
    sel_soup = soup(html, 'html.parser')
    player_box = sel_soup.findAll("div", {"class": "table-scroll"})
    player_box = player_box[0].findAll("tr", {"class": ""})
    for i in range(1,len(player_box)):
        team.append(player_box[i].find("td", {"data-stat": "Tm"}).text)
        hand.append(j)
        games = player_box[i].find("td", {"data-stat": "G"}).text
        singles = player_box[i].find("td", {"data-stat": "1B"}).text
        doubles = player_box[i].find("td", {"data-stat": "2B"}).text
        triples = player_box[i].find("td", {"data-stat": "3B"}).text
        homeruns = player_box[i].find("td", {"data-stat": "HR"}).text
        runs = player_box[i].find("td", {"data-stat": "R"}).text
        RBI = player_box[i].find("td", {"data-stat": "RBI"}).text
        walks = player_box[i].find("td", {"data-stat": "BB"}).text
        stolenbases = player_box[i].find("td", {"data-stat": "SB"}).text
        dkpoints.append(((int(singles) * 3) + (int(doubles)*5) + (int(triples)*8) + (int(homeruns) * 10) + (int(runs)*2) + (int(RBI)*2) + (int(walks)*2) + (int(stolenbases)*5))/(int(games)))

    driver.quit()

team_dk_points_splits_df = pd.DataFrame(
    {
      "Team": team,
      "DKPoints": dkpoints,
      "Hand": hand,
    },
)

In [12]:
team = []
hand = []
hardhitPercentage = []
softhitPercentage = []
fbPercentage = []
gbPercentage = []

for j in range(1,3):
    driver = webdriver.Firefox()
    driver.get(team_battedball.replace("HAND", str(j)))
    html = driver.execute_script("return document.documentElement.outerHTML")
    sel_soup = soup(html, 'html.parser')
    player_box = sel_soup.findAll("div", {"class": "table-scroll"})
    player_box = player_box[0].findAll("tr", {"class": ""})
    for i in range(1,len(player_box)):
        team.append(player_box[i].find("td", {"data-stat": "Tm"}).text)
        hand.append(j)
        hardhitPercentage.append(player_box[i].find("td", {"data-stat": "Hard%"}).text)
        softhitPercentage.append(player_box[i].find("td", {"data-stat": "Soft%"}).text)
        gbPercentage.append(player_box[i].find("td", {"data-stat": "GB%"}).text)
        fbPercentage.append(player_box[i].find("td", {"data-stat": "FB%"}).text)       


    driver.quit()

team_battedball_df = pd.DataFrame(
    {
      "Team": team,
      "Hand": hand,
      "Hard%": hardhitPercentage,
      'Soft%': softhitPercentage,
      'GB%': gbPercentage,
      'FB%': fbPercentage,  
    },
)

In [13]:
team_splits_df = team_advanced_splits_df.merge(team_dk_points_splits_df, right_on = ["Team", "Hand"], left_on = ["Team",  "Hand"], how = "outer").merge(team_battedball_df, right_on = ['Team', 'Hand'], left_on = ["Team",  "Hand"], how='outer')

team_splits_df_lhp_mask = team_splits_df["Hand"] == 1
team_splits_df_lhp = team_splits_df.loc[team_splits_df_lhp_mask,:].copy()

team_splits_df_rhp_mask = team_splits_df["Hand"] == 2
team_splits_df_rhp = team_splits_df.loc[team_splits_df_rhp_mask,:].copy()

In [14]:
team_splits_df_lhp = team_splits_df_lhp.rename(columns={"wRC+": "Team wRC+ vs LHP", "ISO": "Team ISO vs LHP", "DKPoints": "Team DK Points vs LHP", "K%": "Team K% vs LHP", 'Hard%': 'Team Hard% vs LHP', 'Soft%': 'Team Soft% vs LHP', 'FB%': 'Team FB% vs LHP', 'GB%': 'Team GB% vs LHP'}).drop("Hand", axis = 1)
team_splits_df_rhp = team_splits_df_rhp.rename(columns={"wRC+": "Team wRC+ vs RHP", "ISO": "Team ISO vs RHP", "DKPoints": "Team DK Points vs RHP", "K%": "Team K% vs RHP", 'Hard%': 'Team Hard% vs RHP', 'Soft%': 'Team Soft% vs RHP', 'FB%': 'Team FB% vs RHP', 'GB%': 'Team GB% vs RHP'}).drop("Hand", axis = 1)
TeamSplits = team_splits_df_lhp.merge(team_splits_df_rhp, right_on = ["Team"], left_on = ["Team"], how = "outer")

## Pitchers

In [15]:
pitcher_links = [] # List element to put my href links

UClient = UReq(pitcher_dashboard)
page_html = UClient.read()
UClient.close()
page_soup = soup(page_html, "html.parser")
stat_container = page_soup.findAll("td", {"class":"grid_line_regular"})
standard_container = page_soup.findAll("a", {"class":"rtsLink rtsAfter"})
if standard_container[4].has_attr('href'):
        pitcher_links.append(standard_container[4].attrs['href'])

In [16]:
pitcher_standard = "https://www.fangraphs.com/" + pitcher_links[0]  + "&page=1_1000"# Using the extracted href to get to the standard dashboard

In [17]:
pitcher = []
dkpoints = []
player_id = []


UClient = UReq(pitcher_standard)
page_html = UClient.read()
UClient.close()
page_soup = soup(page_html, "html.parser")
stat_container = page_soup.findAll("tr", {"class":"rgRow"})
stat_container2 = page_soup.findAll("tr", {"class": "rgAltRow"})


for i in range(len(stat_container)):
    stat = stat_container[i].findAll("td", {"class", "grid_line_regular"})
    link = stat_container[i].find("a")
    player_id.append(str(link).split('playerid=')[1].split("&")[0])
    pitcher.append(stat[1].text)
    dkpoints.append(((float(stat[13].text)*2.25)+ (float(stat[24].text)*2) + (float(stat[3].text)*4) - (float(stat[17].text)*2) - (float(stat[15].text)*0.6) - (float(stat[19].text)*0.6)-(float(stat[21].text)*0.6))/+ (float(stat[6].text)))
for i in range(len(stat_container2)):
    stat = stat_container2[i].findAll("td", {"class", "grid_line_regular"})
    link = stat_container2[i].find("a")
    player_id.append(str(link).split('playerid=')[1].split("&")[0])
    pitcher.append(stat[1].text)
    dkpoints.append(((float(stat[13].text)*2.25)+ (float(stat[24].text)*2) + (float(stat[3].text)*4) - (float(stat[17].text)*2) - (float(stat[15].text)*0.6) - (float(stat[19].text)*0.6)-(float(stat[21].text)*0.6))/+ (float(stat[6].text)))

In [18]:
pitch_standard_df = pd.DataFrame(
    {
    "Pitcher": pitcher,
    "FGID": player_id,
    "DKPoints": dkpoints,
})

In [19]:
## Toggle to Advanced Page
pitcher_links = []
advanced_container = page_soup.findAll("a", {"class":"rtsLink rtsAfter"})
if advanced_container[4].has_attr('href'):
        pitcher_links.append(advanced_container[4].attrs['href'])

In [20]:
pitcher_advanced = "https://www.fangraphs.com/" + pitcher_links[0]  + "&page=1_1000"# Using the extracted href to get to the advacned dashboard

In [21]:
UClient = UReq(pitcher_advanced)
page_html = UClient.read()
UClient.close()
page_soup = soup(page_html, "html.parser")
stat_container = page_soup.findAll("tr", {"class":"rgRow"})
stat_container2 = page_soup.findAll("tr", {"class": "rgAltRow"})

In [22]:
k9 = []
hr9 = []
player_id = []
for i in range(len(stat_container)):
    stat = stat_container[i].findAll("td", {"class", "grid_line_regular"})
    link = stat_container[i].find("a")
    player_id.append(str(link).split('playerid=')[1].split("&")[0])
    k9.append(stat[3].text)
    hr9.append(stat[6].text)
for i in range(len(stat_container2)):
    stat = stat_container2[i].findAll("td", {"class", "grid_line_regular"})
    link = stat_container2[i].find("a")
    player_id.append(str(link).split('playerid=')[1].split("&")[0])
    k9.append(stat[3].text)
    hr9.append(stat[6].text)

In [23]:
Pitcher_advanced_df = pd.DataFrame(
    {
    "FGID": player_id,
    "K/9": k9,
    "HR9": hr9,
})

In [24]:
## Toggle to batted balls
pitcher_links = []
advanced_container = page_soup.findAll("a", {"class":"rtsLink rtsAfter"})
if advanced_container[4].has_attr('href'):
        pitcher_links.append(advanced_container[4].attrs['href'])

In [25]:
pitcher_battedball = "https://www.fangraphs.com/" + pitcher_links[0]  + "&page=1_1000"# Using the extracted href to get to the batted ball dashboard

In [26]:
UClient = UReq(pitcher_battedball)
page_html = UClient.read()
UClient.close()
page_soup = soup(page_html, "html.parser")
stat_container = page_soup.findAll("tr", {"class":"rgRow"})
stat_container2 = page_soup.findAll("tr", {"class": "rgAltRow"})

In [27]:
hardhit = []
player_id = []
for i in range(len(stat_container)):
    stat = stat_container[i].findAll("td", {"class", "grid_line_regular"})
    link = stat_container[i].find("a")
    player_id.append(str(link).split('playerid=')[1].split("&")[0])
    #pitcher.append(stat[1].text)
    hardhit.append(stat[16].text)
for i in range(len(stat_container2)):
    stat = stat_container2[i].findAll("td", {"class", "grid_line_regular"})
    link = stat_container2[i].find("a")
    player_id.append(str(link).split('playerid=')[1].split("&")[0])
    #pitcher.append(stat[1].text)
    hardhit.append(stat[16].text)

In [28]:
Pitcher_battedballs_df = pd.DataFrame(
    {
    "FGID": player_id,
    "Hard%": hardhit,
})

#### Getting Pitcher Splits

In [29]:
k9 = []
hr9 = []
hand = []
player_id = []

for j in range(5,7):
    driver = webdriver.Firefox()
    driver.implicitly_wait(3)
    driver.get(vs_hand_adv.replace("HAND", str(j)))
    html = driver.execute_script("return document.documentElement.outerHTML")
    sel_soup = soup(html, 'html.parser')
    player_box = sel_soup.findAll("div", {"class": "table-scroll"})
    player_box = player_box[0].findAll("tr", {"class": ""})
    for i in range(1,len(player_box)):
        link = player_box[i].find("a")
        player_id.append(str(link).split('playerid=')[1].split('"')[0])
        k9.append(player_box[i].find("td", {"data-stat": "K/9"}).text)
        hr9.append(player_box[i].find("td", {"data-stat": "HR/9"}).text)
        hand.append(j)
    driver.quit()

In [30]:
pitcher_advanced_splits_df = pd.DataFrame ({
    "FGID": player_id,
    "K/9": k9,
    "HR/9": hr9,
    "Hand": hand,
},
)

In [31]:
hand = []
OPS = []
player_id = []

for j in range(5,7):
    driver = webdriver.Firefox()
    driver.get(vs_hand_std.replace("HAND", str(j)))
    driver.implicitly_wait(3)
    html = driver.execute_script("return document.documentElement.outerHTML")
    sel_soup = soup(html, 'html.parser')
    player_box = sel_soup.findAll("div", {"class": "table-scroll"})
    player_box = player_box[0].findAll("tr", {"class": ""})
    for i in range(1,len(player_box)):
        link = player_box[i].find("a")
        player_id.append(str(link).split('playerid=')[1].split('"')[0])
        hand.append(j)
        games = player_box[i].find("td", {"data-stat": "G"}).text ## Need to divide by IP because they don't have games.... 
        OPS.append(float(player_box[i].find("td", {"data-stat": "OBP"}).text)+float(player_box[i].find("td", {"data-stat": "SLG"}).text))
    driver.quit()


In [32]:
pitcher_standard_splits_df = pd.DataFrame ({
    "FGID": player_id,
    "OPS": OPS,
    "Hand": hand,
},
)

In [33]:
# Merge the two splits together and create the vs LHH and vs RHH columns for each stat
pitcher_splits_df = pitcher_advanced_splits_df.merge(pitcher_standard_splits_df, right_on = ["FGID", "Hand"], left_on = ["FGID", "Hand"], how = "outer")

splits_df_lhh_mask = pitcher_splits_df["Hand"] == 5
splits_df_lhh = pitcher_splits_df.loc[splits_df_lhh_mask,:].copy()

splits_df_rhh_mask = pitcher_splits_df["Hand"] == 6
splits_df_rhh = pitcher_splits_df.loc[splits_df_rhh_mask,:].copy()

pitcher_splits_df_lhh = splits_df_lhh.rename(columns={"OPS": "OPS vs LHH", "K/9": "K/9 vs LHH", "HR/9": "HR/9 vs LHH", "DKPoints": "DK Points vs LHH"}).drop("Hand", axis = 1)
pitcher_splits_df_rhh = splits_df_rhh.rename(columns={"OPS": "OPS vs RHH","K/9": "K/9 vs RHH", "HR/9": "HR/9 vs RHH", "DKPoints": "DK Points vs RHH"}).drop("Hand", axis = 1)
pitcher_splits_df_clean = pitcher_splits_df_lhh.merge(pitcher_splits_df_rhh, right_on = ["FGID"], left_on = ["FGID"], how = "outer")

In [34]:
player_id  = []
l14k9 = []
l14hr9 = []

driver = webdriver.Firefox()
driver.get(pitcher_last_14)
driver.implicitly_wait(3)
html = driver.execute_script("return document.documentElement.outerHTML")
sel_soup = soup(html, 'html.parser')
player_box = sel_soup.findAll("div", {"class": "table-scroll"})
player_box = player_box[0].findAll("tr", {"class": ""})
for i in range(1,len(player_box)):
    link = player_box[i].find("a")
    player_id.append(str(link).split('playerid=')[1].split('"')[0])
    l14k9.append(player_box[i].find("td", {"data-stat": "K/9"}).text)
    l14hr9.append(player_box[i].find("td", {"data-stat": "HR/9"}).text)
driver.quit()

In [35]:
pitcher_l14_df = pd.DataFrame ({
    "FGID": player_id,
    "L14_K/9": l14k9,
    "L14_HR/9": l14hr9,
}
)

In [36]:
for i in range(len(PlayerMap)): #turning my FGID's into strings to merge
    PlayerMap.loc[i,"FGID"] = str(PlayerMap.loc[i,"FGID"])

In [37]:
# Merge all of my pitcher data sets together.
std_adv_merge = pitch_standard_df.merge(Pitcher_advanced_df, right_on = ["FGID"], left_on=["FGID"], how="outer")
std_adv_merge = std_adv_merge.merge(Pitcher_battedballs_df, right_on = ["FGID"], left_on=["FGID"], how="outer")
last14_splits_merge1 = pitcher_l14_df.merge(pitcher_splits_df_clean, right_on=["FGID"], left_on = ["FGID"], how = "outer")
last14_splits_merge = last14_splits_merge1.merge(PlayerMap, right_on="FGID", left_on = "FGID", how = "outer")
the_pitcher_df1 = std_adv_merge.merge(last14_splits_merge, right_on =["FGID"], left_on = ["FGID"], how="inner")
the_pitcher_df1 = the_pitcher_df1.drop_duplicates(subset=['FGID']).reset_index()

In [38]:
opponent = []

for i in range(len(the_pitcher_df1)): #creating an opponent column to merge to opposing team statistics
    if the_pitcher_df1['TeamAbbrev'][i] == the_pitcher_df1['Home Team'][i]:
        opponent.append(the_pitcher_df1['Away Team'][i])
    else:
        opponent.append(the_pitcher_df1['Home Team'][i])
the_pitcher_df1['Opponent'] = opponent

In [39]:
## Fixing some of the different abbresviations:
for i in range(len(TeamSplits)):
    TeamSplits.loc[i,"Team"] = TeamSplits['Team'][i].replace("WSN", "WAS")
    TeamSplits.loc[i,"Team"]= TeamSplits['Team'][i].replace("CHW", "CWS")
    TeamSplits.loc[i,"Team"]= TeamSplits['Team'][i].replace("TBR", "TB")
    TeamSplits.loc[i,"Team"]= TeamSplits['Team'][i].replace("KCR", "KC")
    TeamSplits.loc[i,"Team"]= TeamSplits['Team'][i].replace("SFG", "SF")
    TeamSplits.loc[i,"Team"]= TeamSplits['Team'][i].replace("SDP", "SD")

In [40]:
the_pitcher_df = the_pitcher_df1.merge(TeamSplits, right_on = "Team", left_on = "Opponent", how = "inner")

In [41]:
## Bring in my projected starters and merge their team and name so I can merge to the pitcher_df

player_id = []


UClient = UReq(projected_starters)
page_html = UClient.read()
UClient.close()
page_soup = soup(page_html, "html.parser")
stat_container = page_soup.findAll("tr", {"class":"rgRow"})
stat_container2 = page_soup.findAll("tr", {"class": "rgAltRow"})


for i in range(len(stat_container)):
    stat = stat_container[i].findAll("td", {"class", "grid_line_regular"})
    link = stat_container[i].find("a")
    player_id.append(str(link).split('playerid=')[1].split("&")[0])
for i in range(len(stat_container2)):
    stat = stat_container2[i].findAll("td", {"class", "grid_line_regular"})
    link = stat_container2[i].find("a")
    player_id.append(str(link).split('playerid=')[1].split("&")[0])
  

In [42]:
# player_id.append("13164")
# player_id.append("14975")
# # player_id.append("11682")
# # player_id.append("13431")

# player_id.remove("14078")
# player_id.remove("20418")
# player_id.remove("15488")

In [43]:
pitcher_merge_df = pd.DataFrame(
    {
      "FGID": player_id,
    },
)

In [44]:
### Pitcher dataframe before cleaning
Pitcher_df = the_pitcher_df.merge(pitcher_merge_df, right_on = ["FGID"], left_on = ["FGID"], how = "inner").rename(columns={"Home Team": "Park"})

In [45]:
#### Bring in my park factors and merge to pitchers.... Also, going to use the park factors for the opposite hand of the pitcher with the assumption that opposing teams
### will use more of the opposite hand to stack the pitcher
Park_Factors_Pitchers = Park_Factors.copy()
for i in range(len(Park_Factors_Pitchers)): #switching the hand to be opposite
    if Park_Factors_Pitchers.loc[i,'Hand'] == "R":
        Park_Factors_Pitchers.loc[i,'Hand'] = Park_Factors_Pitchers.loc[i,'Hand'].replace('R', 'L')
    else:
        Park_Factors_Pitchers.loc[i,'Hand'] = Park_Factors_Pitchers.loc[i,'Hand'].replace('L', 'R')
Pitcher_DF = Pitcher_df.merge(Park_Factors_Pitchers, right_on = ["Hand", 'Team Abbrev'], left_on = ['Throws', 'Park'], how = 'inner')

In [46]:
Pitcher_DF.to_excel("Test.xlsx", index=False)

In [47]:
## making columns for the team statistics against the pitchers hand to clean up the ouput sheet. 
## Also, replacing L14 stats with overall stats in case pitcher hasn't played in the last two weeks.
teamwrc = []
teamiso = []
teamdkpoints = []
teamk = []
for i in range(len(Pitcher_DF)):
    if Pitcher_DF.loc[i,'Throws'] == "R":
        teamwrc.append(Pitcher_DF.loc[i,'Team wRC+ vs RHP'])
        teamiso.append(Pitcher_DF.loc[i,'Team ISO vs RHP'])
        teamdkpoints.append(Pitcher_DF.loc[i,'Team DK Points vs RHP'])
        teamk.append(Pitcher_DF.loc[i,'Team K% vs RHP'].replace("%", ""))
    else:
        teamwrc.append(Pitcher_DF.loc[i,'Team wRC+ vs LHP'])
        teamiso.append(Pitcher_DF.loc[i,'Team ISO vs LHP'])
        teamdkpoints.append(Pitcher_DF.loc[i,'Team DK Points vs LHP'])
        teamk.append(Pitcher_DF.loc[i,'Team K% vs LHP'].replace("%", ""))
    if math.isnan(pd.to_numeric(Pitcher_DF.loc[i,"L14_K/9"])) == True:
        Pitcher_DF.loc[i,"L14_K/9"] = Pitcher_DF.loc[i,"K/9"]
    if math.isnan(pd.to_numeric(Pitcher_DF.loc[i,"L14_HR/9"])) == True:
        Pitcher_DF.loc[i,"L14_HR/9"] = Pitcher_DF.loc[i,"HR9"]


Pitcher_DF['Team wRC+'] = teamwrc
Pitcher_DF['Team ISO'] = teamiso
Pitcher_DF['Team DK Points'] = teamdkpoints
Pitcher_DF['Team K%'] = teamk

In [48]:
## Clean the pitchers sheet to be ready for z-scoring:
Pitcher_df_clean1 = Pitcher_DF[["Pitcher", 'FGID', 'Position', 'TeamAbbrev', 'Throws', 'Salary', "DKPoints", 'K/9', 'HR9', 'Hard%', 'L14_K/9', 'L14_HR/9', 'Opponent', 'Park', 'Average Points', 'Team K%', 'Team wRC+', 'Team ISO', 'Team DK Points', 'Park Factors', 'K/9 vs LHH', 'HR/9 vs LHH', 'OPS vs LHH', 'K/9 vs RHH', 'HR/9 vs RHH', 'OPS vs RHH']].rename(columns={'Park Factors': 'Park Factor'})
Pitcher_df_clean1.to_excel("Clean.xlsx", index=False)
pitcher_mask = (Pitcher_df_clean1["Position"] == "SP") | (Pitcher_df_clean1["Position"] == "RP")
Pitcher_df_clean1 = Pitcher_df_clean1.loc[pitcher_mask,:].copy().reset_index().drop("index", axis = 1)

##### Set up my pitcher sheet for the batter projections

In [49]:
pitcher_for_batter = Pitcher_df_clean1[["Pitcher", "FGID", "Position", "TeamAbbrev", "HR9", "Hard%", 'Throws', 'K/9 vs LHH', 'HR/9 vs LHH', 'OPS vs LHH', 'K/9 vs RHH', 'HR/9 vs RHH', 'OPS vs RHH']].rename(columns={"Pitcher": "Opposing Pitcher", "TeamAbbrev": "Opponent"})

# Batters

In [50]:
batter_links = [] # List element to put my href links

UClient = UReq(fg_dashboard)
page_html = UClient.read()
UClient.close()
page_soup = soup(page_html, "html.parser")
stat_container = page_soup.findAll("td", {"class":"grid_line_regular"})
standard_container = page_soup.findAll("a", {"class":"rtsLink rtsAfter"})
if standard_container[4].has_attr('href'):
        batter_links.append(standard_container[4].attrs['href'])

In [51]:
fg_standard = "https://www.fangraphs.com/" + batter_links[0]  + "&page=1_1500"# Using the extracted href to get to the standard dashboard

In [52]:
fg_standard

'https://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=0&type=0&season=2021&month=0&season1=2021&ind=0&team=0&rost=0&age=0&filter=&players=0&startdate=2021-01-01&enddate=2021-12-31&page=1_1500'

In [53]:
batter = []
dkpoints = []
player_id = []
pa = []

UClient = UReq(fg_standard.replace("pos=all", "pos=np"))
page_html = UClient.read()
UClient.close()
page_soup = soup(page_html, "html.parser")
stat_container = page_soup.findAll("tr", {"class":"rgRow"})
stat_container2 = page_soup.findAll("tr", {"class": "rgAltRow"})


for i in range(len(stat_container)):
    stat = stat_container[i].findAll("td", {"class", "grid_line_regular"})
    link = stat_container[i].find("a")
    player_id.append(str(link).split('playerid=')[1].split("&")[0])
    batter.append(stat[1].text)
    games = stat[3].text
    pa.append(int(stat[5].text))
    singles=stat[7].text
    doubles=stat[8].text
    triples=stat[9].text
    homeruns=stat[10].text
    runs=stat[11].text
    RBI=stat[12].text
    walks=stat[13].text
    stolenbases=stat[20].text
    dkpoints.append(((int(singles) * 3) + (int(doubles)*5) + (int(triples)*8) + (int(homeruns) * 10) + (int(runs)*2) + (int(RBI)*2) + (int(walks)*2) + (int(stolenbases)*5))/(int(games)))
for i in range(len(stat_container2)):
    stat = stat_container2[i].findAll("td", {"class", "grid_line_regular"})
    link = stat_container2[i].find("a")
    player_id.append(str(link).split('playerid=')[1].split("&")[0])
    batter.append(stat[1].text)
    games=stat[3].text
    pa.append(int(stat[5].text))
    singles=stat[7].text
    doubles=stat[8].text
    triples=stat[9].text
    homeruns=stat[10].text
    runs=stat[11].text
    RBI=stat[12].text
    walks=stat[13].text
    stolenbases= stat[20].text
    dkpoints.append(((int(singles) * 3) + (int(doubles)*5) + (int(triples)*8) + (int(homeruns) * 10) + (int(runs)*2) + (int(RBI)*2) + (int(walks)*2) + (int(stolenbases)*5))/(int(games)))


In [54]:
standard_df = pd.DataFrame(
    {
    "Batter": batter,
    "FGID": player_id,
    "DKPoints": dkpoints,
    'PA': pa,
})

In [55]:
df_mask = standard_df["PA"] >= np.percentile(standard_df.PA,15)
standard_df = standard_df.loc[df_mask,:].copy().drop(["PA"], axis=1)

In [56]:
batter_links = []

UClient = UReq(fg_standard)
page_html = UClient.read()
UClient.close()
page_soup = soup(page_html, "html.parser")
advanced_container = page_soup.findAll("a", {"class":"rtsLink rtsAfter"})
if advanced_container[4].has_attr('href'):
        batter_links.append(advanced_container[4].attrs['href'])

In [57]:
fg_advanced = "https://www.fangraphs.com/" + batter_links[0]  + "&page=1_1500"# Using the extracted href to get to the advacned dashboard

In [58]:
UClient = UReq(fg_advanced.replace("pos=all", "pos=np"))
page_html = UClient.read()
UClient.close()
page_soup = soup(page_html, "html.parser")
stat_container = page_soup.findAll("tr", {"class":"rgRow"})
stat_container2 = page_soup.findAll("tr", {"class": "rgAltRow"})

In [59]:
wRC = []
ISO = []
player_id = []

for i in range(len(stat_container)):
    stat = stat_container[i].findAll("td", {"class", "grid_line_regular"})
    stat1 = stat_container[i].findAll("td", {"class", "grid_line_break"})
    link = stat_container[i].find("a")
    ISO.append(stat1[1].text)
    wRC.append(stat1[3].text)
    player_id.append(str(link).split('playerid=')[1].split("&")[0])
for i in range(len(stat_container2)):
    stat = stat_container2[i].findAll("td", {"class", "grid_line_regular"})
    stat1 = stat_container2[i].findAll("td", {"class", "grid_line_break"})
    link = stat_container2[i].find("a")
    ISO.append(stat1[1].text)
    wRC.append(stat1[3].text)
    player_id.append(str(link).split('playerid=')[1].split("&")[0])

In [60]:
advanced_df = pd.DataFrame(
    {
     "FGID": player_id,
    "ISO": ISO,
    "wRC+": wRC,
})

#### Pull in lefty and righty splits

In [61]:
wrc = []
ISO = []
hand = []
player_id = []

for j in range(1,3):
    driver = webdriver.Firefox()
    driver.get(vs_hand_website_adv.replace("HAND", str(j)))
    html = driver.execute_script("return document.documentElement.outerHTML")
    sel_soup = soup(html, 'html.parser')
    player_box = sel_soup.findAll("div", {"class": "table-scroll"})
    player_box = player_box[0].findAll("tr", {"class": ""})
    for i in range(1,len(player_box)):
        link = player_box[i].find("a")
        player_id.append(str(link).split('playerid=')[1].split('"')[0])
        wrc.append(player_box[i].find("td", {"data-stat": "wRC+"}).text)
        ISO.append(player_box[i].find("td", {"data-stat": "ISO"}).text)
        hand.append(j)
    driver.quit()

In [62]:
advanced_splits_df = pd.DataFrame ({
    "FGID": player_id,
    "wRC+": wrc,
    "ISO": ISO,
    "Hand": hand,
},
)

In [63]:
hand = []
dkpoints = []
player_id = []

for j in range(1,3):
    driver = webdriver.Firefox()
    driver.get(vs_hand_website_std.replace("HAND", str(j)))
    html = driver.execute_script("return document.documentElement.outerHTML")
    sel_soup = soup(html, 'html.parser')
    player_box = sel_soup.findAll("div", {"class": "table-scroll"})
    player_box = player_box[0].findAll("tr", {"class": ""})
    for i in range(1,len(player_box)):
        hand.append(j)
        link = player_box[i].find("a")
        player_id.append(str(link).split('playerid=')[1].split('"')[0])
        games = player_box[i].find("td", {"data-stat": "G"}).text
        singles = player_box[i].find("td", {"data-stat": "1B"}).text
        doubles = player_box[i].find("td", {"data-stat": "2B"}).text
        triples = player_box[i].find("td", {"data-stat": "3B"}).text
        homeruns = player_box[i].find("td", {"data-stat": "HR"}).text
        runs = player_box[i].find("td", {"data-stat": "R"}).text
        RBI = player_box[i].find("td", {"data-stat": "RBI"}).text
        walks = player_box[i].find("td", {"data-stat": "BB"}).text
        stolenbases = player_box[i].find("td", {"data-stat": "SB"}).text
        dkpoints.append(((int(singles) * 3) + (int(doubles)*5) + (int(triples)*8) + (int(homeruns) * 10) + (int(runs)*2) + (int(RBI)*2) + (int(walks)*2) + (int(stolenbases)*5))/(int(games)))
    driver.quit()

In [64]:
dk_points_splits_df = pd.DataFrame(
    {
      "FGID": player_id,
      "DKPoints": dkpoints,
      "Hand": hand,
    },
)

In [65]:
splits_df = advanced_splits_df.merge(dk_points_splits_df, right_on = ["FGID", "Hand"], left_on = ["FGID", "Hand"], how = "outer")

splits_df_lhp_mask = splits_df["Hand"] == 1
splits_df_lhp = splits_df.loc[splits_df_lhp_mask,:].copy()

splits_df_rhp_mask = splits_df["Hand"] == 2
splits_df_rhp = splits_df.loc[splits_df_rhp_mask,:].copy()

In [66]:
splits_df_lhp = splits_df_lhp.rename(columns={"wRC+": "wRC+ vs LHP", "ISO": "ISO vs LHP", "DKPoints": "DK Points vs LHP"}).drop("Hand", axis = 1)
splits_df_rhp = splits_df_rhp.rename(columns={"wRC+": "wRC+ vs RHP", "ISO": "ISO vs RHP", "DKPoints": "DK Points vs RHP"}).drop("Hand", axis = 1)
splits_df_clean = splits_df_lhp.merge(splits_df_rhp, right_on = ["FGID"], left_on = ["FGID"], how = "inner")

In [67]:
hand = []
dkpoints = []
player_id = []

driver = webdriver.Firefox()
driver.get(last_7_website)
html = driver.execute_script("return document.documentElement.outerHTML")
sel_soup = soup(html, 'html.parser')
player_box = sel_soup.findAll("div", {"class": "table-scroll"})
player_box = player_box[0].findAll("tr", {"class": ""})
for i in range(1,len(player_box)):
    link = player_box[i].find("a")
    player_id.append(str(link).split('playerid=')[1].split('"')[0])
    games = player_box[i].find("td", {"data-stat": "G"}).text
    singles = player_box[i].find("td", {"data-stat": "1B"}).text
    doubles = player_box[i].find("td", {"data-stat": "2B"}).text
    triples = player_box[i].find("td", {"data-stat": "3B"}).text
    homeruns = player_box[i].find("td", {"data-stat": "HR"}).text
    runs = player_box[i].find("td", {"data-stat": "R"}).text
    RBI = player_box[i].find("td", {"data-stat": "RBI"}).text
    walks = player_box[i].find("td", {"data-stat": "BB"}).text
    stolenbases = player_box[i].find("td", {"data-stat": "SB"}).text
    dkpoints.append(((int(singles) * 3) + (int(doubles)*5) + (int(triples)*8) + (int(homeruns) * 10) + (int(runs)*2) + (int(RBI)*2) + (int(walks)*2) + (int(stolenbases)*5))/(int(games)))
driver.quit()

In [68]:
last_7_points_df = pd.DataFrame(
    {
       "FGID": player_id,
       "L7 DKPoints": dkpoints,
    },
)

In [69]:
splits_df_clean.to_excel("splitstest.xlsx", index = False)
for i in range(len(PlayerMap)):
    PlayerMap.loc[i,"FGID"] = str(PlayerMap.loc[i,"FGID"])

#### Bring all of my batter data sets together
#### Also, bring in my daily salaries and game info

In [70]:
std_adv_merge = standard_df.merge(advanced_df, right_on = ["FGID"], left_on=["FGID"], how="outer")
last7_splits_merge = last_7_points_df.merge(splits_df_clean, right_on=["FGID"], left_on = ["FGID"], how = "right")
last7_splits_merge = last7_splits_merge.merge(PlayerMap,right_on=["FGID"], left_on = ["FGID"], how = "outer")
the_batter_df_read = std_adv_merge.merge(last7_splits_merge, right_on ="FGID", left_on = "FGID", how="inner")
the_batter_df_read = the_batter_df_read.drop_duplicates(subset=['FGID']).reset_index()

In [71]:
##Bring in my projected starters and merge their team and name so I can merge to the pitcher_df

opp_pitcher = []
team = []
player_id = []


UClient = UReq(projected_starters)
page_html = UClient.read()
UClient.close()
page_soup = soup(page_html, "html.parser")
stat_container = page_soup.findAll("tr", {"class":"rgRow"})
stat_container2 = page_soup.findAll("tr", {"class": "rgAltRow"})


for i in range(len(stat_container)):
    stat = stat_container[i].findAll("td", {"class", "grid_line_regular"})
    link = stat_container[i].find("a")
    player_id.append(str(link).split('playerid=')[1].split("&")[0])
    opp_pitcher.append(stat[1].text)
    team.append(stat[2].text)
for i in range(len(stat_container2)):
    stat = stat_container2[i].findAll("td", {"class", "grid_line_regular"})
    link = stat_container2[i].find("a")
    player_id.append(str(link).split('playerid=')[1].split("&")[0])
    opp_pitcher.append(stat[1].text)
    team.append(stat[2].text)

In [72]:
# player_id.append("13164")
# player_id.append("14975")
# # player_id.append("13431")

# team.append("BOS")
# team.append("PIT")
# # team.append("NYM")

# opp_pitcher.append("Eduardo Rodriguez")
# opp_pitcher.append("Chad Kuhl")
# opp_pitcher.append("Marcus Stroman")



In [73]:
cleanname = []
for i in range(len(opp_pitcher)):
    cleanname.append(opp_pitcher[i].replace("jr.", "jr").replace(".", "").replace("'", "").replace("ó", "o").replace("é", "e").replace("켈리","").replace("테임즈", "").replace("김광현","").replace("류현진", "").replace("í","i").replace("á", "a").replace("ú", "u").replace("ñ", "n"))

In [74]:
pitcher_merge_df = pd.DataFrame(
    {
    "Opposing Pitcher": cleanname,
    "FGID": player_id,
    },
)

In [75]:
pitcher_merge = pitcher_merge_df.merge(pitcher_for_batter, right_on = "FGID", left_on = "FGID", how="inner").drop(["FGID", "Opposing Pitcher_x"], axis=1).rename(columns={"Opposing Pitcher_y": "Opposing Pitcher"})
pitcher_mask = (pitcher_merge["Position"] == "SP") | (pitcher_merge["Position"] == "RP")
pitcher_merge = pitcher_merge.loc[pitcher_mask,:].copy().reset_index().drop(["index", "Position"], axis = 1)

In [76]:
opponent = []
for i in range(len(the_batter_df_read)):
    if the_batter_df_read['TeamAbbrev'][i] == the_batter_df_read['Home Team'][i]:
        opponent.append(the_batter_df_read['Away Team'][i])
    else:
        opponent.append(the_batter_df_read['Home Team'][i])
the_batter_df_read['Opponent'] = opponent
the_batter_df = the_batter_df_read.merge(pitcher_merge, right_on = [ "Opponent"], left_on = ["Opponent"], how="inner")

In [77]:
#changing the switch hitters to the hand based on the pitcher they are facing
for i in range(len(the_batter_df)):
    if the_batter_df.loc[i,"Bats"] == "B" and the_batter_df.loc[i,"Throws_y"]== "R":
        the_batter_df.loc[i,"Bats"] = the_batter_df.loc[i,"Bats"].replace("B", "L")
    elif the_batter_df.loc[i,"Bats"] == "B" and the_batter_df.loc[i,"Throws_y"]== "L":
        the_batter_df.loc[i,"Bats"] = the_batter_df.loc[i,"Bats"].replace("B", "R")
    elif the_batter_df.loc[i,"Bats"] == "B":
        the_batter_df.loc[i,"Bats"] = the_batter_df.loc[i,"Bats"].replace("B", "R")
        
#change NaN in Last seven points to their total average points
for i in range(len(the_batter_df)):
    if math.isnan(the_batter_df.loc[i,"L7 DKPoints"]) == True:
        the_batter_df.loc[i,"L7 DKPoints"] = the_batter_df.loc[i,"DKPoints"]

#take off the % from hard hit percentage

for i in range(len(the_batter_df)):
    the_batter_df.loc[i,"Hard%"] = the_batter_df.loc[i,"Hard%"].split("%")[0]
for i in range(len(Pitcher_df_clean1['Hard%'])):
    Pitcher_df_clean1.loc[i,"Hard%"] = Pitcher_df_clean1.loc[i,"Hard%"].split("%")[0]

the_batter_df['HR9'] =  the_batter_df['HR9'].fillna(0.98)


In [78]:
Batters = the_batter_df.merge(Park_Factors, right_on = ["Team Abbrev", "Hand"], left_on = ["Home Team", "Bats"], how="left") # bring in the park
Batters_clean = Batters.drop(['Name + ID', "ID", "Roster Position", "Game Info", "Team Abbrev", "Away Team", "Team", "AvgPointsPerGame", "Throws_x", 'Hand', "Average Points", "Park ISO", "Park OPS"], axis = 1).rename(columns={"Park Factors": 'Park Factor', 'Home Team': 'Park', 'Throws_y': 'Pitcher Throws'})

In [79]:
## making columns for the team statistics against the pitchers hand to clean up the ouput sheet. 
## Also, replacing L14 stats with overall stats in case pitcher hasn't played in the last two weeks.
wrcvshand = []
isovshand = []
dkpointsvshand = []
k9vshand = []
hr9vshand = []
opsvshand = []

for i in range(len(Batters_clean)):
    if Batters_clean.loc[i,'Pitcher Throws'] == "R":
        wrcvshand.append(Batters_clean.loc[i,'wRC+ vs RHP'])
        isovshand.append(Batters_clean.loc[i,'ISO vs RHP'])
        dkpointsvshand.append(Batters_clean.loc[i,'DK Points vs RHP'])
    else:
        wrcvshand.append(Batters_clean.loc[i,'wRC+ vs LHP'])
        isovshand.append(Batters_clean.loc[i,'ISO vs LHP'])
        dkpointsvshand.append(Batters_clean.loc[i,'DK Points vs LHP'])


        
for i in range(len(Batters_clean)):
    if Batters_clean.loc[i,'Bats'] == "R":
        k9vshand.append(Batters_clean.loc[i,'K/9 vs RHH'])
        hr9vshand.append(Batters_clean.loc[i,'HR/9 vs RHH'])
        opsvshand.append(Batters_clean.loc[i,'OPS vs RHH'])
    else:
        k9vshand.append(Batters_clean.loc[i,'K/9 vs LHH'])
        hr9vshand.append(Batters_clean.loc[i,'HR/9 vs LHH'])
        opsvshand.append(Batters_clean.loc[i,'OPS vs LHH'])
        
Batters_clean['wRC+ vs Hand'] = wrcvshand
Batters_clean['ISO vs Hand'] = isovshand
Batters_clean['DK Points vs Hand'] = dkpointsvshand
Batters_clean['Pitcher K/9 vs Hand'] = k9vshand
Batters_clean['Pitcher HR/9 vs Hand'] = hr9vshand
Batters_clean['Pitcher OPS vs Hand'] = opsvshand 

In [80]:
## Clean off people who don't have statistics against a certain hand.
Batter_mask = (Batters_clean["wRC+ vs Hand"].isna() == False) & (Batters_clean["Position"] != "RP") & (Batters_clean["Position"] != "SP") & (Batters_clean["Batter"].isna() == False)
Batters_clean = Batters_clean.loc[Batter_mask,:].copy().reset_index().drop("index", axis = 1)

In [81]:
## Clean off people who don't have statistics against a certain hand.
Batter_mask = (Batters_clean["wRC+ vs Hand"].isna() == False) & (Batters_clean["Position"] != "RP") & (Batters_clean["Position"] != "SP") & (Batters_clean["Batter"].isna() == False)
Batters_clean = Batters_clean.loc[Batter_mask,:].copy().reset_index().drop("index", axis = 1)

In [82]:
Batters_clean = Batters_clean[["Batter", 'FGID', 'Salary', 'Position', 'TeamAbbrev', 'Bats', 'ISO', 'wRC+', 'DKPoints', 'L7 DKPoints', 'Opponent', 'Opposing Pitcher', 'Pitcher Throws', 'Pitcher OPS vs Hand', 'Pitcher HR/9 vs Hand', 'HR9', 'Hard%', 'wRC+ vs Hand', 'ISO vs Hand', 'DK Points vs Hand', 'Park', 'Park Factor']] 
for i in range(len(Batters_clean)):
    if Batters_clean.loc[i,'Hard%'] == 'nan':
        Batters_clean.loc[i,'Hard%'] = 25

#### Break up my data into positions

In [83]:
Batters_clean["Website"] = ""

for i in range(len(Batters_clean)):
    Batters_clean.loc[i, 'Website'] = 'https://www.fangraphs.com/players/' + Batters_clean.loc[i,'Batter'].split(" ")[0] + "-" +  Batters_clean.loc[i,'Batter'].split(" ")[1] + "/" + Batters_clean.loc[i,'FGID'] + "/" 
    
Pitcher_df_clean1["Website"] = ""

for i in range(len(Pitcher_df_clean1)):
    Pitcher_df_clean1.loc[i, 'Website'] = 'https://www.fangraphs.com/players/' + Pitcher_df_clean1.loc[i,'Pitcher'].split(" ")[0] + "-" +  Pitcher_df_clean1.loc[i,'Pitcher'].split(" ")[1] + "/" + Pitcher_df_clean1.loc[i,'FGID'] + "/" 

In [84]:
# seveninnning_mask = (Batters_clean["TeamAbbrev"] != "SEA") & (Batters_clean["TeamAbbrev"] != "BAL")
# Batters_clean = Batters_clean.loc[seveninnning_mask,:].copy().reset_index().drop("index", axis = 1)


In [85]:
catchers_mask = (Batters_clean["Position"] == "3B/C") | (Batters_clean["Position"] == "C")
catchers = Batters_clean.loc[catchers_mask,:].copy().reset_index().drop("index", axis = 1)

firstbase_mask = (Batters_clean["Position"] == "1B") | (Batters_clean["Position"] == "1B/2B") | (Batters_clean["Position"] == "1B/3B") | (Batters_clean["Position"] == "1B/OF") 
firstbase = Batters_clean.loc[firstbase_mask,:].copy().reset_index().drop("index", axis = 1)

secondbase_mask = (Batters_clean["Position"] == "2B") | (Batters_clean["Position"] == "1B/2B") | (Batters_clean["Position"] == "2B/3B") | (Batters_clean["Position"] == "2B/OF") | (Batters_clean["Position"] == "2B/SS")
secondbase = Batters_clean.loc[secondbase_mask,:].copy().reset_index().drop("index", axis = 1)

thirdbase_mask = (Batters_clean["Position"] == "3B") | (Batters_clean["Position"] == "2B/3B") | (Batters_clean["Position"] == "1B/3B") | (Batters_clean["Position"] == "3B/OF") | (Batters_clean["Position"] == "3B/C") | (Batters_clean["Position"] == "3B/SS")
thirdbase = Batters_clean.loc[thirdbase_mask,:].copy().reset_index().drop("index", axis = 1)

shortstop_mask = (Batters_clean["Position"] == "SS") | (Batters_clean["Position"] == "2B/SS") | (Batters_clean["Position"] == "3B/SS")  | (Batters_clean["Position"] == "OF/SS")
shortstop = Batters_clean.loc[shortstop_mask,:].copy().reset_index().drop("index", axis = 1)

outfield_mask = (Batters_clean["Position"] == "OF") | (Batters_clean["Position"] == "1B/OF") | (Batters_clean["Position"] == "2B/OF") | (Batters_clean["Position"] == "3B/OF") | (Batters_clean["Position"] == "OF/SS") 
outfield = Batters_clean.loc[outfield_mask,:].copy().reset_index().drop("index", axis = 1)

# Z score my data
#### subsetting by position and exporting the data 

In [86]:
# Z-Score Function
sd = 0
def zscore(my_stat):
    sd = np.std(my_stat)
    avg = statistics.mean(my_stat)
    z_score = (my_stat - avg)/sd
    return z_score

In [87]:
position_list = ['All Batters Z Score', "Catcher", "1B", "2B", "3B", "SS", "OF"]

writer = pd.ExcelWriter('Projection Sheets/ProjectionSheet' + today + "_" + Slate + '.xlsx', engine = 'xlsxwriter')

DKDaily_GameInfo.to_excel(writer, sheet_name = "Game Slate", index=False) ## Game Info
workbook = writer.book
worksheet = writer.sheets["Game Slate"]
worksheet.set_tab_color('#919198')

Batters_clean.to_excel(writer, sheet_name= "Batter Statistics", index = False)  # "Standard Stats"
workbook = writer.book
worksheet = writer.sheets["Batter Statistics"]
worksheet.set_tab_color('#FF9900')

Pitcher_df_clean1.to_excel(writer, sheet_name= "Pitcher Statistics", index = False) # Pitcher Standard Stats
workbook = writer.book
worksheet = writer.sheets["Pitcher Statistics"]
worksheet.set_tab_color('#FF9900')

####PITCHERS#####

zSalary = (zscore(pd.to_numeric(Pitcher_df_clean1['Salary'])) + 3)
zDKPoints = (zscore(pd.to_numeric(Pitcher_df_clean1['DKPoints'])) + 3)
zK9 = (zscore(pd.to_numeric(Pitcher_df_clean1['K/9'])) + 3)
zHR9 = (-zscore(pd.to_numeric(Pitcher_df_clean1['HR9'])) + 3)
zHard = (-zscore(pd.to_numeric(Pitcher_df_clean1['Hard%'])) + 3)
zl14k9 = (zscore(pd.to_numeric(Pitcher_df_clean1['L14_K/9'])) + 3)
zl14hr9 = (-zscore(pd.to_numeric(Pitcher_df_clean1['L14_HR/9'])) + 3)
zparkpts = (-zscore(pd.to_numeric(Pitcher_df_clean1['Park Factor'])) + 3)
zteamK = (zscore(pd.to_numeric(Pitcher_df_clean1['Team K%'])) + 3)
zteamWRC = (-zscore(pd.to_numeric(Pitcher_df_clean1['Team wRC+'])) + 3)
zteamISO = (-zscore(pd.to_numeric(Pitcher_df_clean1['Team ISO'])) + 3)
zteamDKPoints = (-zscore(pd.to_numeric(Pitcher_df_clean1['Team DK Points'])) + 3)

total_z_score = []

for i in range(len(zDKPoints)):
    total_z_score.append((3.5 * (zK9[i] + zteamK[i]))+ (3*(zteamWRC[i] + zparkpts[i]))+(2*(zteamDKPoints[i]+ zteamISO[i]))+ zHR9[i] + zl14k9[i] + zDKPoints[i] + zHard[i] + zl14hr9[i])

total_z = zscore(total_z_score) + 3

pitcher_df = pd.DataFrame ({
    "Pitcher": Pitcher_df_clean1['Pitcher'],
    "FGID": Pitcher_df_clean1['FGID'],
    "TeamAbbrev": Pitcher_df_clean1['TeamAbbrev'],
    'Throws': Pitcher_df_clean1['Throws'],
    'Salary': Pitcher_df_clean1['Salary'],
    'zSalary': zSalary,
    'zDKPoints': zDKPoints,
    'zK/9': zK9,
    'zHR9': zHR9,
    'zHard%': zHard,
    'zL14_K/9': zl14k9,
    'zL14HR/9': zl14hr9,
    'Opponent': Pitcher_df_clean1['Opponent'],
    'Park': Pitcher_df_clean1['Park'],
    'zPark Factor': zparkpts,
    "zTeamK": zteamK,
    "zTeam wRC+": zteamWRC,
    'zTeam ISO': zteamISO,
    'zTeam DK Points': zteamDKPoints,
    "Total Z Score": total_z,
    "Value": total_z/zSalary,
    'Website': Pitcher_df_clean1['Website']
})

pitcher_df.sort_values(by=['Total Z Score'], ascending = False).to_excel(writer, sheet_name= "Pitcher", index = False)

####CATCHERS#####
    
zDKPoints = (zscore(pd.to_numeric(catchers["DKPoints"])) + 3)
zISO = zscore(pd.to_numeric(catchers["ISO"])) + 3
zwRC = zscore(pd.to_numeric(catchers['wRC+'])) + 3
zL7Points = zscore(pd.to_numeric(catchers['L7 DKPoints'])) + 3
zwrcvsHand = zscore(pd.to_numeric(catchers['wRC+ vs Hand'])) + 3
zISOvsHand = zscore(pd.to_numeric(catchers['ISO vs Hand'])) + 3
zDKPointsvsHand = zscore(pd.to_numeric(catchers['DK Points vs Hand'])) + 3 
zSalary = zscore(pd.to_numeric(catchers['Salary'])) + 3
zHR9 = zscore(pd.to_numeric(catchers['HR9'])) + 3
zHardPerc = zscore(pd.to_numeric(catchers['Hard%'])) + 3
zParkPoints = zscore(pd.to_numeric(catchers['Park Factor'])) + 3
zPicherOPSvsHand = zscore(pd.to_numeric(catchers['Pitcher OPS vs Hand'])) + 3
zPitcherHR9vsHand = zscore(pd.to_numeric(catchers['Pitcher HR/9 vs Hand'])) + 3

total_z_score = []
for i in range(len(zDKPoints)):
    total_z_score.append(4*(zISOvsHand[i] + zPicherOPSvsHand[i]) + (3*(zParkPoints[i]+ zPitcherHR9vsHand[i] +zwrcvsHand[i]  + zDKPointsvsHand[i])) + (2*(zDKPoints[i] + zISO[i] + zwRC[i] +  zHR9[i])) + zL7Points[i] + zHardPerc[i])   

total_z = zscore(pd.Series(total_z_score)) + 3

catcher_df = pd.DataFrame ({
    "Batter": catchers["Batter"],
    "FGID": catchers["FGID"],
    "Salary": catchers["Salary"],
    "zSalary": zSalary,
    "Position": catchers["Position"],
    "TeamAbbrev": catchers["TeamAbbrev"],
    "Bats": catchers["Bats"],
    "ISO": zISO,
    "wRC+": zwRC,
    "DKPoints": zDKPoints,
    "L7 DKPoints": zL7Points,
    "Opponent": catchers["Opponent"],
    "Opposing Pitcher": catchers["Opposing Pitcher"],
    "Pitcher Throws": catchers["Pitcher Throws"],
    'Picher OPS vs Hand': zPicherOPSvsHand,
    'Pitcher HR9 vs Hand': zPitcherHR9vsHand,
    "HR9": zHR9,
    "Hard%": zHardPerc,
    "wRC+ vs Hand": zwrcvsHand,
    "ISO vs Hand": zISOvsHand,
    "DKPoints vs Hand": zDKPointsvsHand,
    "Park": catchers["Park"],
    "zPark Factor": zParkPoints,
    "Total Z Score": total_z,
    "Value": total_z/zSalary,
    'Website': catchers['Website']
})


catcher_df.sort_values(by=['Total Z Score'], ascending = False).to_excel(writer, sheet_name= "Catcher", index = False)


####First Basemen#####
    
zDKPoints = zscore(pd.to_numeric(firstbase["DKPoints"])) + 3
zISO = zscore(pd.to_numeric(firstbase["ISO"])) + 3
zwRC = zscore(pd.to_numeric(firstbase['wRC+'])) + 3
zL7Points = zscore(pd.to_numeric(firstbase['L7 DKPoints'])) + 3
zwrcvsHand = zscore(pd.to_numeric(firstbase['wRC+ vs Hand'])) + 3
zISOvsHand = zscore(pd.to_numeric(firstbase['ISO vs Hand'])) + 3
zDKPointsvsHand = zscore(pd.to_numeric(firstbase['DK Points vs Hand'])) + 3
zSalary = zscore(pd.to_numeric(firstbase['Salary'])) + 3
zHR9 = zscore(pd.to_numeric(firstbase['HR9'])) + 3
zHardPerc = zscore(pd.to_numeric(firstbase['Hard%'])) + 3
zParkPoints = zscore(pd.to_numeric(firstbase['Park Factor'])) + 3
zPicherOPSvsHand = zscore(pd.to_numeric(firstbase['Pitcher OPS vs Hand'])) + 3
zPitcherHR9vsHand = zscore(pd.to_numeric(firstbase['Pitcher HR/9 vs Hand'])) + 3


total_z_score = []
for i in range(len(zDKPoints)):
    total_z_score.append(4*(zISOvsHand[i] + zPicherOPSvsHand[i]) + (3*(zParkPoints[i]+ zPitcherHR9vsHand[i] +zwrcvsHand[i]  + zDKPointsvsHand[i])) + (2*(zDKPoints[i] + zISO[i] + zwRC[i] +  zHR9[i])) + zL7Points[i] + zHardPerc[i]) 
    
total_z = zscore(pd.Series(total_z_score)) + 3

firstbase_df = pd.DataFrame ({
    "Batter": firstbase["Batter"],
    "FGID": firstbase["FGID"],
    "Salary": firstbase["Salary"],
    "zSalary": zSalary,
    "Position": firstbase["Position"],
    "TeamAbbrev": firstbase["TeamAbbrev"],
    "Bats": firstbase["Bats"],
    "ISO": zISO,
    "wRC+": zwRC,
    "DKPoints": zDKPoints,
    "L7 DKPoints": zL7Points,
    "Opponent": firstbase["Opponent"],
    "Opposing Pitcher": firstbase["Opposing Pitcher"],
    "Pitcher Throws": firstbase["Pitcher Throws"],
    'Picher OPS vs Hand': zPicherOPSvsHand,
    'Pitcher HR9 vs Hand': zPitcherHR9vsHand,
    "HR9": zHR9,
    "Hard%": zHardPerc,
    "wRC+ vs Hand": zwrcvsHand,
    "ISO vs Hand": zISOvsHand,
    "DKPoints vs Hand": zDKPointsvsHand,
    "Park": firstbase["Park"],
    "Park Factor": zParkPoints,
    "Total Z Score": total_z,
    "Value": total_z/zSalary,
    'Website': firstbase['Website']
})

firstbase_df.sort_values(by=['Total Z Score'], ascending = False).to_excel(writer, sheet_name= "1B", index = False)

####SECONDBASE#####
    
zDKPoints = zscore(pd.to_numeric(secondbase["DKPoints"])) + 3
zISO = zscore(pd.to_numeric(secondbase["ISO"])) + 3
zwRC = zscore(pd.to_numeric(secondbase['wRC+'])) + 3
zL7Points = zscore(pd.to_numeric(secondbase['L7 DKPoints'])) + 3
zwrcvsHand = zscore(pd.to_numeric(secondbase['wRC+ vs Hand'])) + 3
zISOvsHand = zscore(pd.to_numeric(secondbase['ISO vs Hand'])) + 3
zDKPointsvsHand = zscore(pd.to_numeric(secondbase['DK Points vs Hand'])) + 3
zSalary = zscore(pd.to_numeric(secondbase['Salary'])) + 3
zHR9 = zscore(pd.to_numeric(secondbase['HR9'])) + 3
zHardPerc = zscore(pd.to_numeric(secondbase['Hard%'])) + 3
zParkPoints = zscore(pd.to_numeric(secondbase['Park Factor'])) + 3
zPicherOPSvsHand = zscore(pd.to_numeric(secondbase['Pitcher OPS vs Hand'])) + 3
zPitcherHR9vsHand = zscore(pd.to_numeric(secondbase['Pitcher HR/9 vs Hand'])) + 3


total_z_score = []
for i in range(len(zDKPoints)):
    total_z_score.append(4*(zISOvsHand[i] + zPicherOPSvsHand[i]) + (3*(zParkPoints[i]+ zPitcherHR9vsHand[i] +zwrcvsHand[i]  + zDKPointsvsHand[i])) + (2*(zDKPoints[i] + zISO[i] + zwRC[i] +  zHR9[i])) + zL7Points[i] + zHardPerc[i]) 
    
total_z = zscore(pd.Series(total_z_score)) + 3

secondbase_df = pd.DataFrame ({
    "Batter": secondbase["Batter"],
    "FGID": secondbase["FGID"],
    "Salary": secondbase["Salary"],
    "zSalary": zSalary,
    "Position": secondbase["Position"],
    "TeamAbbrev": secondbase["TeamAbbrev"],
    "Bats": secondbase["Bats"],
    "ISO": zISO,
    "wRC+": zwRC,
    "zDKPoints": zDKPoints,
    "L7 DKPoints": zL7Points,
    "Opponent": secondbase["Opponent"],
    "Opposing Pitcher": secondbase["Opposing Pitcher"],
    "Pitcher Throws": secondbase["Pitcher Throws"],
    'Picher OPS vs Hand': zPicherOPSvsHand,
    'Pitcher HR9 vs Hand': zPitcherHR9vsHand,
    "HR9": zHR9,
    "Hard%": zHardPerc,
    "wRC+ vs Hand": zwrcvsHand,
    "ISO vs Hand": zISOvsHand,
    "zDKPoints vs Hand": zDKPointsvsHand,
    "Park": secondbase["Park"],
    "Park Factor": zParkPoints,
    "Total Z Score": total_z,
    "Value": total_z/zSalary,
    'Website': secondbase['Website']
})


secondbase_df.sort_values(by=['Total Z Score'], ascending = False).to_excel(writer, sheet_name= "2B", index = False)

####Third Basemen#####
    
zDKPoints = zscore(pd.to_numeric(thirdbase["DKPoints"])) + 3
zISO = zscore(pd.to_numeric(thirdbase["ISO"])) + 3
zwRC = zscore(pd.to_numeric(thirdbase['wRC+'])) + 3
zL7Points = zscore(pd.to_numeric(thirdbase['L7 DKPoints'])) + 3
zwrcvsHand = zscore(pd.to_numeric(thirdbase['wRC+ vs Hand'])) + 3
zISOvsHand = zscore(pd.to_numeric(thirdbase['ISO vs Hand'])) + 3
zDKPointsvsHand = zscore(pd.to_numeric(thirdbase['DK Points vs Hand'])) + 3
zSalary = zscore(pd.to_numeric(thirdbase['Salary'])) + 3
zHR9 = zscore(pd.to_numeric(thirdbase['HR9'])) + 3
zHardPerc = zscore(pd.to_numeric(thirdbase['Hard%'])) + 3
zParkPoints = zscore(pd.to_numeric(thirdbase['Park Factor'])) + 3
zPicherOPSvsHand = zscore(pd.to_numeric(thirdbase['Pitcher OPS vs Hand'])) + 3
zPitcherHR9vsHand = zscore(pd.to_numeric(thirdbase['Pitcher HR/9 vs Hand'])) + 3


total_z_score = []
for i in range(len(zDKPoints)):
    total_z_score.append(4*(zISOvsHand[i] + zPicherOPSvsHand[i]) + (3*(zParkPoints[i]+ zPitcherHR9vsHand[i] +zwrcvsHand[i]  + zDKPointsvsHand[i])) + (2*(zDKPoints[i] + zISO[i] + zwRC[i] +  zHR9[i])) + zL7Points[i] + zHardPerc[i]) 
    
total_z = zscore(pd.Series(total_z_score)) + 3

thirdbase_df = pd.DataFrame ({
    "Batter": thirdbase["Batter"],
    "FGID": thirdbase["FGID"],
    "Salary": thirdbase["Salary"],
    "zSalary": zSalary,
    "Position": thirdbase["Position"],
    "TeamAbbrev": thirdbase["TeamAbbrev"],
    "Bats": thirdbase["Bats"],
    "ISO": zISO,
    "wRC+": zwRC,
    "zDKPoints": zDKPoints,
    "L7 DKPoints": zL7Points,
    "Opponent": thirdbase["Opponent"],
    "Opposing Pitcher": thirdbase["Opposing Pitcher"],
    "Pitcher Throws": thirdbase["Pitcher Throws"],
    'Picher OPS vs Hand': zPicherOPSvsHand,
    'Pitcher HR9 vs Hand': zPitcherHR9vsHand,
    "HR9": zHR9,
    "Hard%": zHardPerc,
    "wRC+ vs Hand": zwrcvsHand,
    "ISO vs Hand": zISOvsHand,
    "DKPoints vs Hand": zDKPointsvsHand,
    "Park": thirdbase["Park"],
    "Park Factor": zParkPoints,
    "Total Z Score": total_z,
    "Value": total_z/zSalary,
    'Website': thirdbase['Website']
})

thirdbase_df.sort_values(by=['Total Z Score'], ascending = False).to_excel(writer, sheet_name= "3B", index = False)

####SHORTSTOP#####
    
zDKPoints = zscore(pd.to_numeric(shortstop["DKPoints"])) + 3
zISO = zscore(pd.to_numeric(shortstop["ISO"])) + 3
zwRC = zscore(pd.to_numeric(shortstop['wRC+'])) + 3
zL7Points = zscore(pd.to_numeric(shortstop['L7 DKPoints'])) + 3
zwrcvsHand = zscore(pd.to_numeric(shortstop['wRC+ vs Hand'])) + 3
zISOvsHand = zscore(pd.to_numeric(shortstop['ISO vs Hand'])) + 3
zDKPointsvsHand = zscore(pd.to_numeric(shortstop['DK Points vs Hand'])) + 3
zSalary = zscore(pd.to_numeric(shortstop['Salary'])) + 3
zHR9 = zscore(pd.to_numeric(shortstop['HR9'])) + 3
zHardPerc = zscore(pd.to_numeric(shortstop['Hard%'])) + 3
zParkPoints = zscore(pd.to_numeric(shortstop['Park Factor'])) + 3
zPicherOPSvsHand = zscore(pd.to_numeric(shortstop['Pitcher OPS vs Hand'])) + 3
zPitcherHR9vsHand = zscore(pd.to_numeric(shortstop['Pitcher HR/9 vs Hand'])) + 3


total_z_score = []
for i in range(len(zDKPoints)):
    total_z_score.append(4*(zISOvsHand[i] + zPicherOPSvsHand[i]) + (3*(zParkPoints[i]+ zPitcherHR9vsHand[i] +zwrcvsHand[i]  + zDKPointsvsHand[i])) + (2*(zDKPoints[i] + zISO[i] + zwRC[i] +  zHR9[i])) + zL7Points[i] + zHardPerc[i]) 
    
total_z = zscore(pd.Series(total_z_score)) + 3

shortstop_df = pd.DataFrame ({
    "Batter": shortstop["Batter"],
    "FGID": shortstop["FGID"],
    "Salary": shortstop["Salary"],
    "zSalary": zSalary,
    "Position": shortstop["Position"],
    "TeamAbbrev": shortstop["TeamAbbrev"],
    "Bats": shortstop["Bats"],
    "ISO": zISO,
    "wRC+": zwRC,
    "DKPoints": zDKPoints,
    "L7 DKPoints": zL7Points,
    "Opponent": shortstop["Opponent"],
    "Opposing Pitcher": shortstop["Opposing Pitcher"],
    "Pitcher Throws": shortstop["Pitcher Throws"],
    'Picher OPS vs Hand': zPicherOPSvsHand,
    'Pitcher HR9 vs Hand': zPitcherHR9vsHand,
    "HR9": zHR9,
    "Hard%": zHardPerc,
    "wRC+ vs Hand": zwrcvsHand,
    "ISO vs Hand": zISOvsHand,
    "DKPoints vs Hand": zDKPointsvsHand,
    "Park": shortstop["Park"],
    "Park Factor": zParkPoints,
    "Total Z Score": total_z,
    "Value": total_z/zSalary,
    'Website': shortstop['Website']
})


shortstop_df.sort_values(by=['Total Z Score'], ascending = False).to_excel(writer, sheet_name= "SS", index = False)

####Outfield#####
    
zDKPoints = zscore(pd.to_numeric(outfield["DKPoints"])) + 3
zISO = zscore(pd.to_numeric(outfield["ISO"])) + 3
zwRC = zscore(pd.to_numeric(outfield['wRC+'])) + 3
zL7Points = zscore(pd.to_numeric(outfield['L7 DKPoints'])) + 3
zwrcvsHand = zscore(pd.to_numeric(outfield['wRC+ vs Hand'])) + 3
zISOvsHand = zscore(pd.to_numeric(outfield['ISO vs Hand'])) + 3
zDKPointsvsHand = zscore(pd.to_numeric(outfield['DK Points vs Hand'])) + 3
zSalary = zscore(pd.to_numeric(outfield['Salary'])) + 3
zHR9 = zscore(pd.to_numeric(outfield['HR9'])) + 3
zHardPerc = zscore(pd.to_numeric(outfield['Hard%'])) + 3
zParkPoints = zscore(pd.to_numeric(outfield['Park Factor'])) + 3
zPicherOPSvsHand = zscore(pd.to_numeric(outfield['Pitcher OPS vs Hand'])) + 3
zPitcherHR9vsHand = zscore(pd.to_numeric(outfield['Pitcher HR/9 vs Hand'])) + 3


total_z_score = []
for i in range(len(zDKPoints)):
    total_z_score.append(4*(zISOvsHand[i] + zPicherOPSvsHand[i]) + (3*(zParkPoints[i]+ zPitcherHR9vsHand[i] +zwrcvsHand[i]  + zDKPointsvsHand[i])) + (2*(zDKPoints[i] + zISO[i] + zwRC[i] +  zHR9[i])) + zL7Points[i] + zHardPerc[i]) 
    
total_z = zscore(pd.Series(total_z_score)) + 3

outfield_df = pd.DataFrame ({
    "Batter": outfield["Batter"],
    "FGID": outfield["FGID"],
    "Salary": outfield["Salary"],
    "zSalary": zSalary,
    "Position": outfield["Position"],
    "TeamAbbrev": outfield["TeamAbbrev"],
    "Bats": outfield["Bats"],
    "ISO": zISO,
    "wRC+": zwRC,
    "DKPoints": zDKPoints,
    "L7 DKPoints": zL7Points,
    "Opponent": outfield["Opponent"],
    "Opposing Pitcher": outfield["Opposing Pitcher"],
    "Pitcher Throws": outfield["Pitcher Throws"],
    'Picher OPS vs Hand': zPicherOPSvsHand,
    'Pitcher HR9 vs Hand': zPitcherHR9vsHand,
    'HR9': zHR9,
    "Hard%": zHardPerc,
    "wRC+ vs Hand": zwrcvsHand,
    "ISO vs Hand": zISOvsHand,
    "DKPoints vs Hand": zDKPointsvsHand,
    "Park": outfield["Park"],
    "Park Factor": zParkPoints,
    "Total Z Score": total_z,
    "Value": total_z/zSalary,
    'Website': outfield['Website']
})

outfield_df.sort_values(by=['Total Z Score'], ascending = False).to_excel(writer, sheet_name= "OF", index = False)


####All Batters#####
    
zDKPoints = zscore(pd.to_numeric(Batters_clean["DKPoints"])) + 3
zISO = zscore(pd.to_numeric(Batters_clean["ISO"])) + 3
zwRC = zscore(pd.to_numeric(Batters_clean['wRC+'])) + 3
zL7Points = zscore(pd.to_numeric(Batters_clean['L7 DKPoints'])) + 3
zwrcvsHand = zscore(pd.to_numeric(Batters_clean['wRC+ vs Hand'])) + 3
zISOvsHand = zscore(pd.to_numeric(Batters_clean['ISO vs Hand'])) + 3
zDKPointsvsHand = zscore(pd.to_numeric(Batters_clean['DK Points vs Hand'])) + 3
zSalary = zscore(pd.to_numeric(Batters_clean['Salary'])) + 3
zHR9 = zscore(pd.to_numeric(Batters_clean['HR9'])) + 3
zHardPerc = zscore(pd.to_numeric(Batters_clean['Hard%'])) + 3
zParkPoints = zscore(pd.to_numeric(Batters_clean['Park Factor'])) + 3
zPicherOPSvsHand = zscore(pd.to_numeric(Batters_clean['Pitcher OPS vs Hand'])) + 3
zPitcherHR9vsHand = zscore(pd.to_numeric(Batters_clean['Pitcher HR/9 vs Hand'])) + 3


total_z_score = []
for i in range(len(zDKPoints)):
    total_z_score.append(4*(zISOvsHand[i] + zPicherOPSvsHand[i]) + (3*(zParkPoints[i]+ zPitcherHR9vsHand[i] +zwrcvsHand[i]  + zDKPointsvsHand[i])) + (2*(zDKPoints[i] + zISO[i] + zwRC[i] +  zHR9[i])) + zL7Points[i] + zHardPerc[i]) 
    

total_z = zscore(pd.Series(total_z_score)) + 3

Batters_clean_df = pd.DataFrame ({
    "Batter": Batters_clean["Batter"],
    "FGID": Batters_clean["FGID"],
    "Salary": Batters_clean["Salary"],
    "zSalary": zSalary,
    "Position": Batters_clean["Position"],
    "TeamAbbrev": Batters_clean["TeamAbbrev"],
    "Bats": Batters_clean["Bats"],
    "ISO": zISO,
    "wRC+": zwRC,
    "DKPoints": zDKPoints,
    "L7 DKPoints": zL7Points,
    "Opponent": Batters_clean["Opponent"],
    "Opposing Pitcher": Batters_clean["Opposing Pitcher"],
    "Pitcher Throws": Batters_clean["Pitcher Throws"],
    'Picher OPS vs Hand': zPicherOPSvsHand,
    'Pitcher HR9 vs Hand': zPitcherHR9vsHand,
    "HR9": zHR9,
    "Hard%": zHardPerc,
    "wRC+ vs Hand": zwrcvsHand,
    "ISO vs Hand": zISOvsHand,
    "DKPoints vs Hand": zDKPointsvsHand,
    "Park": Batters_clean["Park"],
    "Park Factor": zParkPoints,
    "Total Z Score": total_z,
    "Value": total_z/zSalary,
    'Website': Batters_clean['Website']
})

Batters_clean_df.sort_values(by=['Total Z Score'], ascending = False).to_excel(writer, sheet_name= "All Batters Z Score", index = False)


cond_list = ['H', 'I', 'J', 'K', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'W', 'X', 'Y', 'Z']

cond_list1 = ['G', 'H', 'I', 'J', 'K', 'L', 'O', 'P', 'Q', 'R', 'S', 'T', 'U']

for i in position_list:
    workbook = writer.book
    worksheet = writer.sheets[i]
    worksheet.set_tab_color('green')
    
    worksheet.conditional_format('D2:D' + str(len(Batters_clean_df)+1), {'type': '2_color_scale',
                                       'min_color': '#33FF00',
                                       'mid_color': "#FFFFFF",
                                       "max_color": '#FF0000'})
    for j in cond_list:
            worksheet.conditional_format(j+'2:' + j + str(len(Batters_clean_df)+1), {'type': '2_color_scale',
                                       'min_color': '#FF0000',
                                       'mid_color': "#FFFFFF",
                                       "max_color": '#33FF00'})
    
    
workbook = writer.book
worksheet = writer.sheets["Pitcher"]
worksheet.set_tab_color('green')

worksheet.conditional_format('E2:E' + str(len(Pitcher_df_clean1)+1), {'type': '2_color_scale',
                                   'min_color': '#33FF00',
                                   'mid_color': "#FFFFFF",
                                   "max_color": '#FF0000'})

for j in cond_list1:
        worksheet.conditional_format(j+'2:' + j + str(len(Pitcher_df_clean1)+1), {'type': '2_color_scale',
                                   'min_color': '#FF0000',
                                   'mid_color': "#FFFFFF",
                                   "max_color": '#33FF00'})
writer.save()
writer.close()